# req

> Fill in a module description here

In [ ]:
#| default_exp req

In [ ]:
#| export
import requests
from pathlib import Path
from html2text import HTML2Text  
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import traceback
from scraper.core import *
from lxml.html import fromstring, HtmlElement, tostring
from lxml_html_clean import Cleaner

h2t = HTML2Text(bodywidth=1000_000)
h2t.ignore_links = True
h2t.mark_code = True
h2t.ignore_images = True
cleaner = Cleaner(javascript=True, style=True)

In [ ]:
class Req:
    def __init__(self, url):
        self.url = url
        self.fn = get_fn_from_url(url)
        self.loc_domain = urlparse(self.url).netloc
        self.dir = None

    def html2md(self):
        try:
            resp = requests.get(self.url, timeout=100)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            
            body = cleaner.clean_html(fromstring(str(soup)).xpath('//body')[0])

            assert isinstance(body, HtmlElement)

            with open(self.dir/f"{self.fn.split('.')[0]}.md", 'w', encoding='utf-8') as f:
                    f.write(h2t.handle(tostring(body, encoding='unicode')))

            self.links = [link for link in [hydrate_links(self.loc_domain, a['href']) 
                                            for a in soup.find_all('a', href=True) 
                                            if valid_href(a['href'])] if link]

        except Exception as e:
            traceback.print_exception(type(e), e, e.__traceback__)
    
    def download_file(self):
        """
        Downloads a file from the specified URL and saves it to the given path
        """
        try:
            resp = requests.get(self.url)
            if resp.status_code == 200:
                with open(self.dir/self.fn, 'wb') as f:
                    f.write(resp.content)
        except Exception as e:
            traceback.print_exc()
    
    def __call__(self, dir:Path):
        self.dir = dir/self.loc_domain
        self.dir.mkdir(parents=True, exist_ok=True)
        f_ext = '.'+self.fn.split('.')[-1]

        if f_ext == '.html':
            self.html2md()
        elif f_ext in ['.pdf', '.doc', '.docx', '.odt', '.xls', '.xlsx', '.ppt', '.pptx', '.txt', '.csv']:
            self.download_file()
        else:
            print(f"could not process {self.url}")


In [ ]:
r = Req('https://mazagondock.in/')
r(Path('./Text'))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()